In [1]:
import csv
import numpy as np
import pandas as pd
import sys, os
import re
import ast
import datetime as dt
from sklearn.preprocessing import MultiLabelBinarizer

In [10]:
diag = pd.read_csv('../data/hos/diagnoses_icd.csv').reset_index(drop = True)
# diag = diag.reset_index(inplace=True)
diag

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9
...,...,...,...,...,...
4756321,19999987,23865745,7,41401,9
4756322,19999987,23865745,8,78039,9
4756323,19999987,23865745,9,0413,9
4756324,19999987,23865745,10,36846,9


In [13]:
diag['new'] = diag['icd_code']

In [19]:

if diag.icd_version == 9:
    diag['new']= '9_' + diag['new'].value
else:
    diag['new']= '10_' +diag['new'].value
return diag['new']

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [7]:
d_diag = pd.read_csv('../data/hos/d_icd_diagnoses.csv').reset_index(drop = True)
# diag = diag.reset_index(inplace=True)
d_diag.head()

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A


In [11]:
(diag
 .merge(d_diag, how='inner', left_on=['icd_code', 'icd_version'], right_on=['icd_code', 'icd_version'])
.reset_index())

,index,subject_id,hadm_id,seq_num,icd_code,icd_version,long_title
0,0,10000032,22595853,1,5723,9,Portal hypertension
1,1,10000826,20032235,4,5723,9,Portal hypertension
2,2,10000826,28289260,1,5723,9,Portal hypertension
3,3,10005866,26158160,4,5723,9,Portal hypertension
4,4,10008924,23676183,7,5723,9,Portal hypertension
...,...,...,...,...,...,...,...
4756321,4756321,19990427,29695607,24,T24011A,10,"Burn of unspecified degree of right thigh, ini..."
4756322,4756322,19996016,28015466,4,O30093,10,"Twin pregnancy, unable to determine number of ..."
4756323,4756323,19996783,25894657,20,H353131,10,"Nonexudative age-related macular degeneration,..."
4756324,4756324,19997062,20096107,1,K8036,10,Calculus of bile duct with acute and chronic c...
